In [2]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import time
import pymongo

In [3]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [18]:
url = 'https://ark.intel.com/content/www/us/en/ark.html#@Processors'
baseurl = 'https://ark.intel.com/'

browser.visit(url)

rootHTML = bs(browser.html, 'html.parser')

In [5]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
db = client.IntelScrape
collection = db.items

In [19]:
for link in bs(browser.html, 'html.parser').\
    find_all('div', {'data-parent-panel-key':'PanelLabel122139'})[0].\
    find_all('a'):
    browser.visit(baseurl+link['href'])
    time.sleep(1)
    for td in bs(browser.html, 'html.parser').find_all('td', class_='ark-product-name'):
        browser.visit(baseurl+td.a['href'])
        time.sleep(1)
        data = {}
        for c in bs(browser.html, 'html.parser').\
            find_all('div', class_='arkProductSpecifications')[0].\
            find_all('span', class_=['label','value']):
            if c['class'][0] == 'label':
                temp = c.text.strip().replace('.','_')
            else:
                data[temp] = c.text.strip()
        collection.insert_one(data)
        print(data['Processor Number'], end=' ')

i9-10980XE i9-10940X i9-10920X i9-10900X i9-9980XE i9-9960X i9-9940X i9-9920X i9-9900X i9-9820X i7-9800X i9-7980XE i9-7960X i9-7940X i9-7920X i9-7900X i7-7820X i7-7800X i7-7740X i5-7640X i7-6950X i7-6900K i7-6850K i7-6800K i7-5960X i7-5930K i7-5820K i7-4960X i7-4940MX i7-4930K i7-4930MX i7-4820K i7-3970X i7-3960X i7-3940XM i7-3930K i7-3920XM i7-3820 i7-1065G7 i7-1060G7 i7-10710U i7-10510U i7-10510Y i5-1035G7 i5-1030G7 i5-1035G4 i5-1030G4 i5-1035G1 i5-10210U i5-10310Y i5-10210Y i3-1000G4 i3-1005G1 i3-1000G1 i3-10110U i3-10110Y i9-9980HK i9-9880H i9-9900T i9-9900KF i9-9900K i9-9900 i7-9850HL i7-9850HE i7-9850H i7-9750HF i7-9750H i7-9700TE i7-9700T i7-9700KF i7-9700K i7-9700F i7-9700E i7-9700 i5-9600T i5-9600K i5-9600KF i5-9600 i5-9500TE i5-9500T i5-9500F i5-9500E i5-9500 i5-9400F i5-9400T i5-9400H i5-9400 i5-9300H i3-9350KF i3-9350K i3-9320 i3-9300T i3-9300 i3-9100TE i3-9100T i3-9100HL i3-9100F i3-9100E i3-9100 i9-8950HK i7-8665UE i7-8665U i7-8557U i7-8850H i7-8809G i7-8750H i7-8709G i7-

In [21]:
browser.quit()

In [42]:
import csv
with open('IntelArk.csv','w',newline='') as csvfile:
    cols = []
    data = db.items.find()
    for i in data:
        cols.extend([x for x in i.keys() if x not in cols])
    cols.remove('_id')
    
    writer = csv.DictWriter(csvfile, cols)
    writer.writeheader()
    for i in data:
        i.pop('_id')
        writer.writerow(i)